### Антипов Д. С.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

In [10]:
# Загрузка датасета
data = pd.read_csv('../test.csv')

X = data.drop(columns=["is_canceled"])
y = data["is_canceled"]

### PCA

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

RANDOM_SEED = 47

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

pca = PCA(n_components=X.shape[1])
principal_components = pca.fit_transform(X)

data = pd.DataFrame(data=principal_components)

### KNN

In [12]:
# Предсказание is_canceled

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3003)

# Создание и обучение модели KNN
knn = KNeighborsClassifier(n_neighbors=12, weights='distance')
knn.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = knn.predict(X_test)
y_pred_proba = knn.predict_proba(X_test)[:, 1]

# Оценка модели
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC_AUC: ", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.8206391875746715

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.92      0.88      9438
           1       0.76      0.58      0.66      3954

    accuracy                           0.82     13392
   macro avg       0.80      0.75      0.77     13392
weighted avg       0.81      0.82      0.81     13392

ROC_AUC:  0.8488391829197457
